# Extraction et structuration des données à partir du journal clinique d'oncologie en utilisant les techniques du NLP

<br>
<div>

<img src="https://github.com/MaartenGr/KeyBERT/assets/25746895/7351a3f1-f3a4-4911-8535-664f37adec78" width="750"/>
</div>


## Installation du package transformers à partir du dépôt GitHub de Hugging Face.🤗

In [1]:
!pip install git+https://github.com/huggingface/transformers -q peft accelerate bitsandbytes sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
from google.colab import drive
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import os
import json

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/LLMS/Data' #le chemin du dossier où se trouvent les journaux sur Google Drive.



Mounted at /content/drive


# Modèle 1 : **QAmembert-large**


Info about ['QAmembert-large'](https://huggingface.co/CATIE-AQ/QAmembert-large)

In [5]:
model_name = "CATIE-AQ/QAmembert-large"

questions = [
    "Quel est le nom et le prénom  du patient ?",
    "Quel âge a le patient ?",
    "Quel est le genre du patient ?",
    "Quels symptômes le patient présente-t-il ?",
    "Quels antécédents médicaux le patient a-t-il ?",
    "Quels médicaments le patient prend-il actuellement ?",
    "Quel diagnostic a été établi pour le patient ?",
    "Quels tests ont été effectués pour parvenir au diagnostic ?",
    "Quelle est la situation actuelle du patient (par exemple, admis à l'hôpital, en cours de traitement, etc.) ?",
]

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)


results = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r') as file:
            content = file.read()

        answers = [nlp({'question': q, 'context': content})['answer'] for q in questions]

        patient_info = {
            "name": answers[0] if answers[0] else "NULL",
            "age": answers[1] if answers[1] else "NULL",
            "gender": answers[2] if answers[2] else "NULL",
            "clinical_observations": answers[3] if answers[3] else "NULL",
            "medical_history": answers[4] if answers[4] else "NULL",
            "current_medications": answers[5].split(", ") if answers[5] else [],
            "diagnosis": answers[6] if answers[6] else "NULL",
            "tests_performed": answers[7] if answers[7] else "NULL",
            "admission_status": answers[8] if answers[8] else "NULL"
        }

        result = {"filename": filename, "patient_info": patient_info}

        results.append(result)

json_results = json.dumps(results, indent=4)

print(json_results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

[
    {
        "filename": "1.txt",
        "patient_info": {
            "name": "Jean Dupont,",
            "age": " 55",
            "gender": " 55 ans,",
            "clinical_observations": " douleurs thoraciques et des difficult\u00e9s respiratoires.",
            "medical_history": " d'hypertension art\u00e9rielle",
            "current_medications": [
                " metformine,"
            ],
            "diagnosis": " diab\u00e9tique",
            "tests_performed": " Un \u00e9lectrocardiogramme",
            "admission_status": " Il a \u00e9t\u00e9 admis \u00e0 l'h\u00f4pital pour une prise en charge suppl\u00e9mentaire."
        }
    },
    {
        "filename": "2.txt",
        "patient_info": {
            "name": "Marie Martin,",
            "age": " 45 ans,",
            "gender": " femme",
            "clinical_observations": " douleurs abdominales persistantes et des naus\u00e9es.",
            "medical_history": " reflux gastro-\u0153sophagien",
            "cur

# Modèle 2 : mdeberta-v3-base-squad2

Info about ['mdeberta-v3-base-squad2'](https://huggingface.co/timpal0l/mdeberta-v3-base-squad2)

In [6]:
model_name = "timpal0l/mdeberta-v3-base-squad2"

questions = [
    "Quel est le nom et le prénom  du patient ?",
    "Quel âge a le patient ?",
    "Quel est le genre du patient ?",
    "Quels symptômes le patient présente-t-il ?",
    "Quels antécédents médicaux le patient a-t-il ?",
    "Quels médicaments le patient prend-il actuellement ?",
    "Quel diagnostic a été établi pour le patient ?",
    "Quels tests ont été effectués pour parvenir au diagnostic ?",
    "Quelle est la situation actuelle du patient (par exemple, admis à l'hôpital, en cours de traitement, etc.) ?",
]

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)


results = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()

        answers = [nlp({'question': q, 'context': content})['answer'] for q in questions]

        patient_info = {
            "name": answers[0] if answers[0] else "NULL",
            "age": answers[1] if answers[1] else "NULL",
            "gender": answers[2] if answers[2] else "NULL",
            "clinical_observations": answers[3] if answers[3] else "NULL",
            "medical_history": answers[4] if answers[4] else "NULL",
            "current_medications": answers[5].split(", ") if answers[5] else [],
            "diagnosis": answers[6] if answers[6] else "NULL",
            "tests_performed": answers[7] if answers[7] else "NULL",
            "admission_status": answers[8] if answers[8] else "NULL"
        }

        result = {"filename": filename, "patient_info": patient_info}

        results.append(result)

json_results = json.dumps(results, indent=4,ensure_ascii=False)

print(json_results.encode('utf8').decode('utf8'))



config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

[
    {
        "filename": "1.txt",
        "patient_info": {
            "name": "Jean Dupont,",
            "age": " 55 ans,",
            "gender": " 55 ans,",
            "clinical_observations": " douleurs thoraciques et des difficultés respiratoires.",
            "medical_history": " d'hypertension artérielle",
            "current_medications": [
                " la metformine",
                "le lisinopril et l'atorvastatine."
            ],
            "diagnosis": " diabétique",
            "tests_performed": " électrocardiogramme",
            "admission_status": " douleurs thoraciques et des difficultés respiratoires."
        }
    },
    {
        "filename": "2.txt",
        "patient_info": {
            "name": "Marie Martin,",
            "age": " 45",
            "gender": " 45 ans,",
            "clinical_observations": " douleurs abdominales persistantes et des nausées.",
            "medical_history": " reflux gastro-œsophagien",
            "current_medicatio

# Modèle 3 : bert-base-multilingual-cased-finetuned-squad

Info about [:bert-base-multilingual-cased-finetuned-squad](https://huggingface.co/salti/bert-base-multilingual-cased-finetuned-squad)
  


In [7]:
model_name = "salti/bert-base-multilingual-cased-finetuned-squad"


questions = [
    "Quel est le nom et le prénom  du patient ?",
    "Quel âge a le patient ?",
    "Quel est le genre du patient ?",
    "Quels symptômes le patient présente-t-il ?",
    "Quels antécédents médicaux le patient a-t-il ?",
    "Quels médicaments le patient prend-il actuellement ?",
    "Quel diagnostic a été établi pour le patient ?",
    "Quels tests ont été effectués pour parvenir au diagnostic ?",
    "Quelle est la situation actuelle du patient (par exemple, admis à l'hôpital, en cours de traitement, etc.) ?",
]

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)


results = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()

        answers = [nlp({'question': q, 'context': content})['answer'] for q in questions]

        patient_info = {
            "name": answers[0] if answers[0] else "NULL",
            "age": answers[1] if answers[1] else "NULL",
            "gender": answers[2] if answers[2] else "NULL",
            "clinical_observations": answers[3] if answers[3] else "NULL",
            "medical_history": answers[4] if answers[4] else "NULL",
            "current_medications": answers[5].split(", ") if answers[5] else [],
            "diagnosis": answers[6] if answers[6] else "NULL",
            "tests_performed": answers[7] if answers[7] else "NULL",
            "admission_status": answers[8] if answers[8] else "NULL"
        }

        result = {"filename": filename, "patient_info": patient_info}

        results.append(result)

json_results = json.dumps(results, indent=4,ensure_ascii=False)

print(json_results.encode('utf8').decode('utf8'))




config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[
    {
        "filename": "1.txt",
        "patient_info": {
            "name": "Jean Dupont",
            "age": "55 ans",
            "gender": "diabétique",
            "clinical_observations": "douleurs thoraciques et des difficultés respiratoires",
            "medical_history": "la metformine, le lisinopril",
            "current_medications": [
                "lisinopril et l'atorvastatine"
            ],
            "diagnosis": "diabétique",
            "tests_performed": "Un électrocardiogramme",
            "admission_status": "une prise en charge supplémentaire"
        }
    },
    {
        "filename": "2.txt",
        "patient_info": {
            "name": "Marie Martin",
            "age": "45",
            "gender": "chirurgien",
            "clinical_observations": "douleurs abdominales persistantes et des nausées",
            "medical_history": "l'oméprazole et le dicyclomine",
            "current_medications": [
                "l'oméprazole et le dicyclomine"


# Modèle 4 : xlm-roberta-large-squad2
Info about [xlm-roberta-large-squad2](https://huggingface.co/deepset/xlm-roberta-large-squad2)


# Modèle 4 : filipealmeida/Mistral-7B-Instruct-v0.1-sharded
Info about [filipealmeida/Mistral-7B-Instruct-v0.1-sharded](https://huggingface.co/filipealmeida/Mistral-7B-Instruct-v0.1-sharded)



In [ ]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'filipealmeida/Mistral-7B-Instruct-v0.1-sharded'

def load_quantized_model(model_name: str):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1
    return tokenizer


model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

stop_token_ids = [0]



`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST] How AI will replace Engineers [/INST] It is possible that

It is possible that AI may be used to automate certain tasks that are typically done by engineers. For example, AI may be used to analyze data and provide recommendations to engineers, or to assist with the design of products or systems. This could potentially lead to some jobs in engineering being eliminated, at least in certain areas. However, AI is not likely to replace all engineers anytime soon. Engineers will still be needed to oversee the development and implementation of new technologies, and to ensure that these technologies are safe and effective. Additionally, as with any new technology, there will likely be unforeseen challenges and complexities that cannot be handled by AI alone, requiring the expertise of engineers.

In conclusion, AI will replace engineers to some extent but, they won't be fully replaced as engineers will still be needed to oversee the development and implementation of new technologies.</s